### ASSDM之总流程

#### 1.提取高层特征
---

In [1]:
import torch as t
from config import DefaultConfig
from tqdm import tqdm_notebook
import torch.nn as nn
opt = DefaultConfig()
model = t.load('checkpoints/0509_21:33:01.pth')

In [2]:
model

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
    (2): ReLU(inplace)
    (3): Dropout(p

In [3]:
list(model.children())[-1][:3]

Sequential(
  (0): Dropout(p=0.5)
  (1): Linear(in_features=9216, out_features=4096, bias=True)
  (2): ReLU(inplace)
)

In [4]:
class Net(nn.Module):
    def __init__(self , model):
        super(Net, self).__init__()
        #取掉model的后两层
        self.fature = nn.Sequential(*list(model.children())[:-1])
        self.first_fc = nn.Sequential(*list(model.children())[-1][:3])
    def forward(self, x):
        x = self.fature(x)
        return x

In [5]:
new_model=Net(model)

In [6]:
from data.dataset import UCMdata
from torch.utils.data import DataLoader
from torch.autograd import Variable
train_data = UCMdata(opt.train_data_root, train=True, start=0, end=0.1)
val_data = UCMdata(opt.train_data_root, train=False, start=80, end=100)
train_dataloader = DataLoader(train_data,
                              1,
                              shuffle=True,
                              num_workers=opt.num_workers)
val_dataloader = DataLoader(val_data,
                            opt.batch_size,
                            shuffle=False,
                            num_workers=opt.num_workers)



In [7]:
for ii,(data,label) in enumerate(tqdm_notebook(train_dataloader,desc='111')):
    print(data.size(),label)
    result = new_model(data)
    print(result.size())

1
0
torch.Size([1, 3, 227, 227]) tensor([0])
torch.Size([1, 256, 6, 6])
torch.Size([1, 3, 227, 227]) tensor([1])
torch.Size([1, 256, 6, 6])

